In [1]:
import torch
import torchvision    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


ModuleNotFoundError: No module named 'torch'

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

In [ ]:
print(train_data)

In [ ]:
print(test_data)


In [ ]:
print(train_data.data.size())


In [ ]:
print(train_data.targets.size())

In [ ]:
#plot one train data
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('tkAgg')
fig = plt.figure()
fig.set_tight_layout(True)

plt.imshow(train_data.data[0], cmap='gray')
plt.title('%i' % train_data.targets[0])

plt.show()
#print(plt.show())

In [ ]:
#plot multiple train_data
figure  = plt.figure(figsize=(10,8))
cols, rows = 5,5 
for i in range(1, cols * rows +1):
    sample_idx = torch.randint(len(train_data), size=(1,) ).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i )
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
#prepare data for training with DataLoaders
# use function to get data been shuffled to prevent overfitting

from torch.utils.data import DataLoader

loaders = { 
    'train' :torch.utils.data.DataLoader(train_data, batch_size=100, 
                                         shuffle= True, num_workers=1),
    'test': torch.utils.data.DataLoader(test_data,batch_size = 100,
                                       shuffle=True, num_workers=1),
}
loaders

In [ ]:
#define the cnn model 
#use 2 fully convolutional layers
#relu activation function and Mappooling
import torch.nn as nn 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization
    
    # The forward() pass defines the way we compute ouroutput using the given 
    #layers and functions

    


In [ ]:
cnn = CNN()
print(cnn)

In [ ]:
#defile a loss function 
loss_func = nn.CrossEntropyLoss()
loss_func


In [ ]:
from torch import optim 

optimizer = optim.Adam(cnn.parameters(), lr = 0.01)
optimizer

In [ ]:
from torch.autograd import Variable
num_epochs = 10
def train(num_epochs, cnn, loaders):
    
    cnn.train()
        
    # Train the model
    total_step = len(loaders['train'])
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                pass
        
            pass
        pass
train(num_epochs, cnn, loaders)
        
    

In [ ]:
def test():
    # Test the model
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loaders['test']:
            test_output, last_layer = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            pass
    print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    
    pass
test()

In [ ]:
sample = next(iter(loaders['test']))
imgs, lbls = sample



In [ ]:
actual_number  = lbls[:10].numpy()
print(actual_number)

In [ ]:
#test_output, last_layer = cnn(imgs[:10])
#pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
#print(f'Prediction number: {pred_y}')
#print(f'Actual number: {actual_number}')

In [ ]:
#import onnx to the program and output the result 

import torch.onnx 
from torch import nn 

# the dummy_imput shoule be the size of imput (train data),  1x1x28x28 means black and white picture have 28*28 pixel shape
dummy_input = torch.randn(1,1,28,28)


torch.onnx.export(cnn, dummy_input, "output.onnx", verbose=True)
